1. Veri seti okuma

In [122]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif
import numpy as np

df = pd.read_csv("datasets/thyroid_cancer_risk_data.csv")

# İlgili olmayan sütunları çıkar (ID, ülke bilgisi, risk seviyesi)
df_cleaned = df.drop(columns=["Patient_ID", "Country", "Ethnicity", "Thyroid_Cancer_Risk"])
df.head()

,Patient_ID,Age,Gender,Country,Ethnicity,Family_History,Radiation_Exposure,Iodine_Deficiency,Smoking,Obesity,Diabetes,TSH_Level,T3_Level,T4_Level,Nodule_Size,Thyroid_Cancer_Risk,Diagnosis
0,1,66,Male,Russia,Caucasian,No,Yes,No,No,No,No,9.37,1.67,6.16,1.08,Low,Benign
1,2,29,Male,Germany,Hispanic,No,Yes,No,No,No,No,1.83,1.73,10.54,4.05,Low,Benign
2,3,86,Male,Nigeria,Caucasian,No,No,No,No,No,No,6.26,2.59,10.57,4.61,Low,Benign
3,4,75,Female,India,Asian,No,No,No,No,No,No,4.10,2.62,11.04,2.46,Medium,Benign
4,5,35,Female,Germany,African,Yes,Yes,No,No,No,No,9.10,2.11,10.71,2.11,High,Benign


2. Hedef değişkeni sayısallaştırma

In [123]:
# Diagnosis: Benign → 0, Malignant → 1
df_cleaned["Diagnosis"] = df_cleaned["Diagnosis"].map({"Benign": 0, "Malignant": 1})

3. Kategorik sütunlara Label Encoder uygulama

In [124]:
# Nesne tipindeki sütunları bul
categorical_cols = df_cleaned.select_dtypes(include="object").columns

# Her biri için LabelEncoder uygula
le = LabelEncoder()
for col in categorical_cols:
    df_cleaned[col] = le.fit_transform(df_cleaned[col])

# Dönüştürülmüş veri çerçevesi
df_cleaned.head()

,Age,Gender,Family_History,Radiation_Exposure,Iodine_Deficiency,Smoking,Obesity,Diabetes,TSH_Level,T3_Level,T4_Level,Nodule_Size,Diagnosis
0,66,1,0,1,0,0,0,0,9.37,1.67,6.16,1.08,0
1,29,1,0,1,0,0,0,0,1.83,1.73,10.54,4.05,0
2,86,1,0,0,0,0,0,0,6.26,2.59,10.57,4.61,0
3,75,0,0,0,0,0,0,0,4.10,2.62,11.04,2.46,0
4,35,0,1,1,0,0,0,0,9.10,2.11,10.71,2.11,0


4. Özellik ve hedefi ayırma

In [125]:
# Bağımsız değişkenler (X) ve hedef değişken (y)
X = df_cleaned.drop(columns=["Diagnosis"])
y = df_cleaned["Diagnosis"]

5. Eksik değerleri doldurma

In [126]:
# Sayısal ve kategorik sütunları ayır
numeric_cols = X.select_dtypes(include=["float64", "int64"]).columns
categorical_cols = X.columns.difference(numeric_cols)

# Sayısal: ortalama ile doldur
X[numeric_cols] = X[numeric_cols].apply(lambda col: col.fillna(col.mean()))

# Kategorik: mod (en sık) ile doldur
X[categorical_cols] = X[categorical_cols].apply(lambda col: col.fillna(col.mode()[0]))

print("✅ Eksik değerler başarıyla temizlendi.")

✅ Eksik değerler başarıyla temizlendi.


6. Sayısal verilerin ölçeklendirilmesi (Standart Scaler)

In [127]:
scaler = StandardScaler()
X_scaled_array = scaler.fit_transform(X)

# Tekrar DataFrame'e çevir
X_scaled = pd.DataFrame(X_scaled_array, columns=X.columns)

7. Eğitim ve test seti ayırma

In [128]:
# Eğitim ve test setlerini oluştur
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Boyutları kontrol et
print("Eğitim kümesi:", X_train.shape)
print("Test kümesi:", X_test.shape)

Eğitim kümesi: (170152, 12)
Test kümesi: (42539, 12)


8. SelectKBest ile en iyi özellik seçimi

In [129]:
# K en iyi özelliği seçeceğiz (örneğin 10)
selector = SelectKBest(score_func=f_classif, k=10)
X_train_selected = selector.fit_transform(X_train, y_train)

# Aynı sütunları test verisine uygula
X_test_selected = selector.transform(X_test)

# Seçilen sütun isimlerini al (isteğe bağlı)
selected_features = X.columns[selector.get_support()]
print("Seçilen özellikler:", list(selected_features))

Seçilen özellikler: ['Age', 'Family_History', 'Radiation_Exposure', 'Iodine_Deficiency', 'Smoking', 'Diabetes', 'TSH_Level', 'T3_Level', 'T4_Level', 'Nodule_Size']


8. SMOTE ile veri dengeleme

In [130]:
# SMOTE sonrası veriyi dengele
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train, y_train)

# SelectKBest ile en iyi 10 özelliği seç
selector = SelectKBest(score_func=f_classif, k=10)
X_train_selected = selector.fit_transform(X_train_resampled, y_train_resampled)

# Seçilen sütunlar
selected_features = X_train.columns[selector.get_support()]

print("✅ SMOTE sonrası veri dengelendi.")
print(y_train_resampled.value_counts())

✅ SMOTE sonrası veri dengelendi.
Diagnosis
1    130581
0    130581
Name: count, dtype: int64


9. Ön işlenmiş veriyi kaydetme

In [131]:
# Doğru sütunlarla test verisini kaydet
test_df = pd.concat([pd.DataFrame(X_test_selected, columns=selected_features), y_test.reset_index(drop=True)], axis=1)
train_df = pd.concat([pd.DataFrame(X_train_resampled, columns=selected_features), y_train_resampled.reset_index(drop=True)], axis=1)

# Verileri kaydedelim
train_df.to_csv("datasets/processed_train_data.csv", index=False)
test_df.to_csv("datasets/processed_test_data.csv", index=False)

print("✅ Doğru eğitim ve test verisi kaydedildi.")

✅ Doğru eğitim ve test verisi kaydedildi.
